# <center>How to systematically search keywords on municipal websites</center>

**Cite as**: Cai, M., Huang, H., & Decaminada, T. (2023). Local data at a national scale: Introducing a dataset of official municipal websites in the United States for text-based analytics. Environment and Planning B: Urban Analytics and City Science, 50(7), 1988-1993. https://doi.org/10.1177/23998083231190961

This tutorial demonstrates how to systematically search self-defined keywords on municipal websites using the dataset UScityURL.csv and [Google Custom Search API](https://developers.google.com/custom-search/v1/introduction), and save the search results as a csv file. Google offers 100 queries per day for free. Daily quotas reset at midnight Pacific Time. Additional queries cost $5 per 1000 queries. Details about Google API pricing could be find [here](https://developers.google.com/custom-search/v1/overview).

First, please set up a Google API key and case ID following the [instructions](https://support.google.com/googleapi/answer/6158862?hl=en). And then, in the same folder where you run code, save the API key in a txt file and name it `api_key.txt`. The txt file should only have the key as its content. Similarly, save the case ID in a txt file and name it `case_id.txt`. These are unique identifiers to authenticate and authorize access to the Google Search API. It is important to remember that they should never be shared with others.

Next, the Google API client library needs to be installed. The documentation is available [here](https://pypi.org/project/google-api-python-client/). After successful install, import all the libraries.

In [1]:
## remove the # in the next line to install the Google API client library
#!pip install google-api-python-client

In [4]:
# import libraries
import pandas as pd
from googleapiclient.discovery import build
import csv
import math
import time

In [5]:
# read in your API key and case ID
api_key = open('api_key.txt', 'r').read().strip()
case_id = open('case_id.txt', 'r').read().strip()

After the above preparations, we are ready for the search.

In [6]:
# load in the dataset and check it out
UScityURL = pd.read_csv("UScityURL.csv") # make sure data and notebook in the same folder otherwise change path
UScityURL.head()

,GEOID,MUNICIPALITY,STATE,WEBSITE_AVAILABLE,WEBSITE_URL
0,1600000US3651000,New York,New York,1,https://www.nyc.gov/
1,1600000US0644000,Los Angeles,California,1,https://www.lacity.org/
2,1600000US1714000,Chicago,Illinois,1,https://www.chicago.gov/
3,1600000US4835000,Houston,Texas,1,http://www.houstontx.gov/
4,1600000US0455000,Phoenix,Arizona,1,https://www.phoenix.gov/


A code book `codebook.txt` for the dataset is available in the repo. 

As of the time when the dataset is compiled (September 2022), 13,724 out of 19,518 municipalities (70%) have an official website. All the municipalities without official websites have populations below 6,000.

In [4]:
UScityURL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19518 entries, 0 to 19517
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   GEOID              19518 non-null  object
 1   MUNICIPALITY       19518 non-null  object
 2   STATE              19518 non-null  object
 3   WEBSITE_AVAILABLE  19518 non-null  int64 
 4   WEBSITE_URL        13724 non-null  object
dtypes: int64(1), object(4)
memory usage: 762.5+ KB


In [7]:
# a ready-to-use function for conducting systematic searches
def search(search_term, api_key, case_id):
    """
    Use Google Custom Search API to systematically search self-defined keywords.
    
    Arguments:
        search_term: search string. The maximum length is 2048 characters.
        api_key: your api key.
        case_id: your case id.
    Returns:
        len(link_list): the number of search results returned.
        title_list: the titles of returned search results in a list.
        link_list: the links of returned search results in a list.
        snippet_list: the snippets of returned search results in a list.
    """
    
    service = build("customsearch", "v1", developerKey=api_key)
    result = service.cse().list(q=search_term, cx=case_id).execute()
    est_total_num = int(result["searchInformation"]["totalResults"])
    title_list = []
    link_list = []
    snippet_list = []
    if est_total_num == 0:
        return len(link_list), title_list, link_list, snippet_list
    elif est_total_num <= 10:
        for item in result["items"]:
            title_list.append(item["title"])
            link_list.append(item["link"])
            snippet_list.append(item['snippet'])
        return len(link_list), title_list, link_list, snippet_list
    else:
        for item in result["items"]:
            title_list.append(item["title"])
            link_list.append(item["link"])
            snippet_list.append(item['snippet'])
        total_page = math.ceil(est_total_num/10)
        if total_page > 10:
            total_page = 10
        for page in range(1, total_page):
            start = page * 10 + 1
            more_result = service.cse().list(q=search_term, cx=case_id, start=start).execute()
            new_total_num = int(more_result["searchInformation"]["totalResults"])
            if new_total_num == 0:
                return len(link_list), title_list, link_list, snippet_list
            else:
                for item in more_result["items"]:
                    title_list.append(item["title"])
                    link_list.append(item["link"])
                    snippet_list.append(item['snippet'])
        return len(link_list), title_list, link_list, snippet_list

In [8]:
# define your search keyword
keyword = "pizza"

In [9]:
# give your output file a name
output_file = "output_example.csv"

In [10]:
# filter out municipalities without websites
source = UScityURL[UScityURL.WEBSITE_AVAILABLE==1].reset_index(drop=True)

In [11]:
# conduct the search and write into a csv file
for i in range(0, 5): # run 5 cities as an example
    search_term = keyword + ' site:' + source.WEBSITE_URL.iloc[i]
    total, title, link, snippet = search(search_term, api_key, case_id)
    csv.writer(open(output_file, "a")).writerow([source.GEOID.iloc[i], total, title, link, snippet])
    time.sleep(1) # to avoid too many requests error

In [12]:
# check the output file
output = pd.read_csv(output_file, names=["GEOID","total","title","link","snippet"])

In [13]:
output

,GEOID,total,title,link,snippet
0,1600000US3651000,73,['As First-Ever Citywide Ranked Choice Voting ...,['https://www.nyc.gov/office-of-the-mayor/news...,"[""As First-Ever Citywide Ranked Choice Voting ..."
1,1600000US0644000,0,[],[],[]
2,1600000US1714000,100,['LICENSE APPEAL COMMISSION CITY OF CHICAGO 20...,['https://www.chicago.gov/content/dam/city/dep...,"['Knipper testified 2033 Pizza, Incorporated, ..."
3,1600000US4835000,80,['How to dispose of or recycle Pizza Box (Grea...,['https://www.houstontx.gov/solidwaste/htxcoll...,['Pizza Box (Greasy). Yard Waste Collections i...
4,1600000US0455000,57,"['Restaurant Stormwater Pollution - Phoenix', ...",['https://www.phoenix.gov/waterservicessite/Do...,"[""Pouring liquid fat, cooking oil, grease, or ..."


The returned search results pinpoint where a certain topic is discussed. They are suitable for further text-based analysis.

In [15]:
# to check out the titles of the returned search results
print(output.title.iloc[2])

['LICENSE APPEAL COMMISSION CITY OF CHICAGO 2033 Pizza ...', 'PUBLIC PLACE OF AMUSEMENT LICENSE PLAN OF OPERATION ...', "AGREED PLAN OF OPERATION Licensee: Barraco's Pizza, Inc. d/b ...", 'CITY OF CHICAGO COMMISSION ON HUMAN RELATIONS IN THE ...', 'Taste of Chicago 2023 – Grant Park Food Vendor Menus', 'City of Chicago COMMISSION ON HUMAN RELATIONS IN THE ...', "What Can I Recycle? Fact Sheet Don't forget! Flatten cardboard and ...", "Mayor Emanuel Welcomes Food Network's Great ... - City of Chicago", 'DAILY HEARING SCHEDULE', 'Save Time And Skip The Line – Contactless ... - City of Chicago', 'Food Scrap Drop-Off', 'Taste of Chicago Grant Park - City of Chicago', 'MEMORANDUM', '6/14/2012 Mount Greenwood SSA District Plan', 'Untitled', 'Divvy Celebrates Fourth Birthday On Wednesday - City of Chicago', 'Taste Grant Park Map & Schedule 2023', 'Taste of Chicago Grant Park Map 2023 (PDF)', 'Restaurant, Banquet Hall Approved For Former ... - City of Chicago', 'Community Development Grant Nov